In [2]:
import pandas as pd 
import matplotlib as plt
import numpy as np
import sklearn
import psycopg2 as psy
import csv
from sqlalchemy import create_engine
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
%matplotlib inline
from sklearn.decomposition import PCA 
import pandas as pd
from pandas.io import sql
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from matplotlib import cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn import metrics


In [35]:
cancel = pd.read_csv("/Users/EKandTower/GA-DSI/projects/projects-weekly/project-07/\
assets/airport_cancellations.csv")

ops = pd.read_csv("/Users/EKandTower/GA-DSI/projects/projects-weekly/project-07/\
assets/Airport_operations.csv")

ap = pd.read_csv("/Users/EKandTower/GA-DSI/projects/projects-weekly/project-07/\
assets/airports.csv")

cancel.rename(columns ={'Airport':'AP_NAME'}, inplace=True)

df = cancel.join(ap, rsuffix='ap')
df.pop('AP_NAMEap');

df = df.join(ops, rsuffix='ops')

df['FAA REGION'] = LabelEncoder().fit_transform(df['FAA REGION'])
df['LocID'] = LabelEncoder().fit_transform(df['LocID'])

df['ALIAS'] = LabelEncoder().fit_transform(df['ALIAS'])
df['Facility Type'] = LabelEncoder().fit_transform(df['Facility Type'])
df['COUNTY'] = LabelEncoder().fit_transform(df['COUNTY'])
df['CITY'] = LabelEncoder().fit_transform(df['CITY'])
df['STATE'] = LabelEncoder().fit_transform(df['STATE'])
df['AP Type'] = LabelEncoder().fit_transform(df['AP Type'])
df['Boundary Data Available'] = LabelEncoder().fit_transform(df['Boundary Data Available'])
df['airport'] = LabelEncoder().fit_transform(df['airport'])



,0,1,2,3,4,5,6,7,8,9,...,795,796,797,798,799,800,801,802,803,804
ap_name,ABQ,ABQ,ABQ,ABQ,ABQ,ABQ,ABQ,ABQ,ABQ,ABQ,...,TUS,TUS,TUS,TUS,TUS,TUS,TUS,TUS,TUS,TUS
year,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
D_cancel,242,221,392,366,333,192,222,345,237,232,...,126,206,281,206,117,144,204,117,175,178
A_cancel,235,190,329,304,300,162,177,294,181,212,...,101,174,237,174,100,110,183,107,153,155
D_divert,71,61,71,107,79,74,71,77,65,62,...,36,63,65,59,38,42,43,38,62,55
A_divert,46,33,124,45,42,26,50,61,60,46,...,26,46,22,16,28,35,28,29,18,39
fac_type,Airport,Airport,Airport,Airport,Airport,Airport,Airport,Airport,Airport,Airport,...,Airport,Airport,Airport,Airport,Airport,Airport,Airport,Airport,Airport,Airport
faa,ASO,ASO,ASO,ASO,ASO,ASO,ASO,ASO,ASO,ASO,...,AWP,ASW,AWP,ASW,ASO,ASW,AWP,ASO,ASW,ASO
state,VI,PR,PR,PR,PR,PR,PR,PR,PR,VI,...,AZ,TX,AZ,TX,AL,TX,AZ,SC,NM,MS
ap_type,Public Use,Public Use,Public Use,Public Use,Public Use,Public Use,Public Use,Public Use,Public Use,Public Use,...,Public Use,Public Use,Public Use,Public Use,Public Use,Public Use,Public Use,Public Use,Federalized/Commercial,Public Use


In [36]:
df_d = pd.get_dummies(df.loc[:, ['fac_type','faa','state', 'ap_type']])

df = pd.concat([df, df_d], axis=1)

df.drop(['fac_type','faa','state', 'ap_type'],axis=1, inplace=True)


df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)

df.head()


df.T.head(100)


,0,1,2,3,4,5,6,7,8,9,...,795,796,797,798,799,800,801,802,803,804
ap_name,ABQ,ABQ,ABQ,ABQ,ABQ,ABQ,ABQ,ABQ,ABQ,ABQ,...,TUS,TUS,TUS,TUS,TUS,TUS,TUS,TUS,TUS,TUS
year,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
D_cancel,242,221,392,366,333,192,222,345,237,232,...,126,206,281,206,117,144,204,117,175,178
A_cancel,235,190,329,304,300,162,177,294,181,212,...,101,174,237,174,100,110,183,107,153,155
D_divert,71,61,71,107,79,74,71,77,65,62,...,36,63,65,59,38,42,43,38,62,55
A_divert,46,33,124,45,42,26,50,61,60,46,...,26,46,22,16,28,35,28,29,18,39
lat,17.7016,17.9822,18.0083,18.1348,18.138,18.2453,18.2557,18.308,18.3133,18.3373,...,33.2691,33.2692,33.2778,33.2813,33.284,33.2929,33.2989,33.3004,33.3016,33.3035
long,-64.8017,-66.0193,-66.563,-65.4936,-65.8007,-65.6434,-67.1485,-65.6618,-65.3043,-64.9734,...,-111.811,-97.4519,-111.127,-97.4967,-85.809,-95.8964,-111.915,-80.3943,-104.531,-89.2284
D_4met,53971,51829,49682,53255,49589,42550,42302,42283,39105,35277,...,4935,4863,28711,29219,29286,30326,26049,20989,22793,24396
A_4met,53818,51877,51199,53611,49512,42704,42169,42297,39107,34989,...,4873,4871,28964,29332,29520,30630,25870,20866,22898,24798


In [52]:
# Adding 5 year change columns for all 

df['Chg_D_cancel'] = df['D_cancel'].pct_change()
df['Chg_A_cancel'] = df['A_cancel'].pct_change()
df['Chg_D_divert'] = df['D_divert'].pct_change()
df['Chg_A_divert'] = df['A_divert'].pct_change()
df['Chg_D_4met'] = df['D_4met'].pct_change()
df['Chg_A_4met'] = df['A_4met'].pct_change()
df['Chg_%on/t_gate_D'] = df['%on/t_gate_D'].pct_change()
df['Chg_%on/t_ap_D'] = df['%on/t_ap_D'].pct_change()
df['Chg_%on/t_gate_A'] = df['%on/t_gate_A'].pct_change()
df['Chg_avg_taxi_out_time'] = df['avg_taxi_out_time'].pct_change()
df['Chg_avg_taxi_out_del'] = df['avg_taxi_out_del'].pct_change()
df['Chg_avg_ap_D_del'] = df['avg_ap_D_del'].pct_change()
df['Chg_avg_airborne_del'] = df['avg_airborne_del'].pct_change()
df['Chg_avg_taxi_in_del'] = df['avg_taxi_in_del'].pct_change()
df['Chg_avg_block_del'] = df['avg_block_del'].pct_change()
df['Chg_avg_gate_A_del'] = df['avg_gate_A_del'].pct_change()

df.T.head(100)

,5,6,7,8,9,10,11,12,13,14,...,793,794,795,796,797,798,799,800,801,803
ap_name,ABQ,ABQ,ABQ,ABQ,ABQ,ABQ,ANC,ANC,ANC,ANC,...,TPA,TUS,TUS,TUS,TUS,TUS,TUS,TUS,TUS,TUS
year,2009,2010,2011,2012,2013,2014,2004,2005,2006,2007,...,2014,2004,2005,2006,2007,2008,2009,2010,2011,2013
D_cancel,192,222,345,237,232,315,473,424,367,359,...,723,103,126,206,281,206,117,144,204,175
A_cancel,162,177,294,181,212,274,532,484,436,423,...,691,80,101,174,237,174,100,110,183,153
D_divert,74,71,77,65,62,66,73,74,88,93,...,183,50,36,63,65,59,38,42,43,62
A_divert,26,50,61,60,46,37,30,31,50,54,...,163,22,26,46,22,16,28,35,28,18
lat,18.2453,18.2557,18.308,18.3133,18.3373,18.4393,18.4511,18.4568,18.4949,19.7203,...,33.259,33.2608,33.2691,33.2692,33.2778,33.2813,33.284,33.2929,33.2989,33.3016
long,-65.6434,-67.1485,-65.6618,-65.3043,-64.9734,-66.002,-66.6756,-66.0985,-67.1294,-155.048,...,-116.321,-103.278,-111.811,-97.4519,-111.127,-97.4967,-85.809,-95.8964,-111.915,-104.531
D_4met,42550,42302,42283,39105,35277,33292,16503,72043,75708,78741,...,5546,5546,4935,4863,28711,29219,29286,30326,26049,22793
A_4met,42704,42169,42297,39107,34989,33137,15511,67034,69995,73323,...,5444,5474,4873,4871,28964,29332,29520,30630,25870,22898


In [53]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)



In [43]:
# df.hist(figsize=(20,20))

In [54]:
x = df.ix[:,2:].values
y = df.ix[:,0:2].values

In [55]:
bad_indices = np.where(np.isinf(x))

print(bad_indices)

(array([], dtype=int64), array([], dtype=int64))


In [56]:
xStand = StandardScaler().fit_transform(x)
covMat = np.cov(xStand.T)

eigenValues, eigenVectors = np.linalg.eig(covMat)

In [57]:
xStand = StandardScaler().fit_transform(x)
covMat = np.cov(xStand.T)

eigenValues, eigenVectors = np.linalg.eig(covMat)

In [58]:
eigenPairs = [(np.abs(eigenValues[i]), eigenVectors[:,i]) for i in range(len(eigenValues))]
eigenPairs.sort()
eigenPairs.reverse()
# for i in eigenPairs:
#     print(i[0]);

In [59]:
totalEigen = sum(eigenValues)
varExpl = [(i / totalEigen)*100 for i in sorted(eigenValues, reverse=True)]
cumulvarExpl = np.cumsum(varExpl)

print(cumulvarExpl)

[  15.37424039   25.13147128   32.97743536   39.60493351   45.51991289
   50.97417144   55.94584584   59.80673341   63.41765227   66.57872289
   68.82610724   71.04128722   73.09680575   75.09816136   77.09386282
   79.03571824   80.93985193   82.805083     84.64635806   86.46691909
   88.2048188    89.92347119   91.44073413   92.7685212    93.83993217
   94.60787565   95.3105739    95.9732247    96.56234831   97.09876089
   97.57060632   97.98061353   98.28641108   98.58965513   98.86135594
   99.09288188   99.31257544   99.49593308   99.66477741   99.78845639
   99.87348851   99.93347233   99.95452183   99.97482101   99.98732095
   99.99431218   99.99734006   99.99904351   99.99977362  100.          100.
  100.          100.          100.          100.          100.        ]
